# 24: LA REGRESIÓN LOGÍSTICA

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Math # esta libreria sirve para mostrar en pantalla fórmulas matemáticas

La regresion logística es aquella que busca predecir un valor categórico en lugar de un valor numérico utilizando las propiedades del logaritmo neperiano (Ln) ya que se traza una función que va de 0 a 1 pasando por 0'5. Si la función supera el 0'5, se considera que hay más probabilidad de estar en la situación 1, en cmabio si estamos más cerca de 0 se considera que es más probable la condición 0.

Fórmula de la regresión logística:

In [2]:
display(Math(r' P = \frac{1}{1+e^{-(\alpha+\beta\cdot X)}}'))

<IPython.core.display.Math object>

In [3]:
data = pd.read_csv('../../Recursos_curso/datasets/gender-purchase/Gender Purchase.csv')

In [4]:
data.head()

,Gender,Purchase
0,Female,Yes
1,Female,Yes
2,Female,No
3,Male,No
4,Male,Yes


### Datos del dataframe

In [5]:
# Visualizamos un resumen de los datos mediante una tabla de contingencia para ver cuantos casos tenemos de cada clase

tabla_contingencia = pd.crosstab(data['Gender'], # Ponemos la primera clase
                                 data['Purchase']) # Ponemos la segunda clase
tabla_contingencia

Purchase,No,Yes
Gender,,
Female,106,159
Male,125,121


In [6]:
# Podemos sumar los diferentes contajes a partir de la tabla de contingencia

females_totales = tabla_contingencia.sum(axis = 1).iloc[0]
males_totales = tabla_contingencia.sum(axis = 1).iloc[1]
no_totales = tabla_contingencia.sum(axis = 0).iloc[0]
yes_totales = tabla_contingencia.sum(axis = 0).iloc[1]

In [7]:
print(f'El número de FEMALES es: {females_totales}')
print(f'El número de MALES es: {males_totales}')
print(f'El número de NO es: {no_totales}')
print(f'El número de YES es: {yes_totales}')

El número de FEMALES es: 265
El número de MALES es: 246
El número de NO es: 231
El número de YES es: 280


In [8]:
# Visualizar la proporción de compras en función de cada género

tabla_contingencia.astype('float').div(tabla_contingencia.sum(axis = 1), # el primer argumento define el valor que dividie
                                       axis = 0) # el segundo argumento define como se hace la division en el dataframe

Purchase,No,Yes
Gender,,
Female,0.40000,0.60000
Male,0.50813,0.49187


In [9]:
# Hacemos lo mismo pero en función de quien compra mas o menos

tabla_contingencia.astype('float').div(tabla_contingencia.sum(axis = 0), axis = 1)

Purchase,No,Yes
Gender,,
Female,0.458874,0.567857
Male,0.541126,0.432143


### Probabilidades condicionales

Las probabilidades condicionales definen cuales son las diferentes probabilidades de que pase un suceso siempre y cuando tambien pase otro suceso. En este caso tenemos las siguientes posibles probabilidades condicionales (las mismas que hemos visto en la tabla de contingencia):

- Probabilidad de ser HOMBRE y COMPRAR
- Probabilidad de ser HOMBRE y NO COMPRAR

- Probabilidad de ser MUJER y COMPRAR
- Probabilidad de ser MUJER y NO COMPRAR

In [21]:
# Probabilidad de ser hombre y comprar

display(Math(r'P(Purchase|Male) = \frac{Numero\ de\ compras\ hechas\ por\ hombres}{Numero\ de\ hombres\ del\ grupo}'))
compra_hombre = tabla_contingencia.iloc[1, 1]
prob_hombre_comprar = compra_hombre / males_totales
print(f'La probabilidad de ser hombre y comprar es: {prob_hombre_comprar}')

<IPython.core.display.Math object>

La probabilidad de ser hombre y comprar es: 0.491869918699187


In [22]:
# Probabilidad de ser hombre y NO comprar

display(Math(r'P(Purchase|Male) = \frac{Numero\ de\ compras\ NO\ hechas\ por\ hombres}{Numero\ de\ hombres\ del\ grupo}'))
no_compra_hombre = tabla_contingencia.iloc[1, 0]
prob_hombre_no_comprar = no_compra_hombre / males_totales
print(f'La probabilidad de ser hombre y comprar es: {prob_hombre_no_comprar}')

<IPython.core.display.Math object>

La probabilidad de ser hombre y comprar es: 0.508130081300813


In [23]:
# Probabilidad de ser mujer y comprar

display(Math(r'P(Purchase|Female) = \frac{Numero\ de\ compras\ hechas\ por\ mujer}{Numero\ de\ mujer\ del\ grupo}'))
compra_mujer = tabla_contingencia.iloc[0, 1]
prob_mujer_comprar = compra_mujer / females_totales
print(f'La probabilidad de ser mujer y comprar es: {prob_mujer_comprar}')

<IPython.core.display.Math object>

La probabilidad de ser mujer y comprar es: 0.6


In [25]:
# Probabilidad de ser mujer y NO comprar

display(Math(r'P(Purchase|Female) = \frac{Numero\ de\ compras\ NO\ hechas\ por\ mujer}{Numero\ de\ mujer\ del\ grupo}'))
no_compra_mujer = tabla_contingencia.iloc[0, 0]
prob_mujer_no_comprar = no_compra_mujer / females_totales
print(f'La probabilidad de ser mujer y NO comprar es: {prob_mujer_no_comprar}')

<IPython.core.display.Math object>

La probabilidad de ser mujer y NO comprar es: 0.4


### Ratio de probabilidades

El ratio de probabilidades es el coeficiente entre la probabilidad de casos de éxito sobre los de fracaso para cada uno de los grupos estudiados.

* Si el ratio es superior a 1, es más probable el éxito que el fracas. Cuanto mayor es el ratio, más probabilidad de éxito en nuestro suceso.
* Si el ratio es exactamente igual a 1, éxito y fracaso son equiprobables (p=0.5)
* Si el ratio es menor que 1, el fracaso es más probable que el éxito. Cuanto menor es el ratio, menor es la probabilidad de éxito del suceso.

In [30]:
display(Math(r'RATIO_{compra,\ mujer} = \frac{Probabilidad\ compra\ mujer}{1 - Probabilidad\ compra\ mujer}'))

display(Math(r'RATIO_{compra,\ hombre} = \frac{Probabilidad\ compra\ hombre}{1 - Probabilidad\ compra\ hombre}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [31]:
# Coeficiente compra respecto a mujeres

coef_mujer = prob_mujer_comprar / (1 - prob_mujer_comprar)
coef_mujer

1.4999999999999998

In [32]:
# Coeficiente compra respecto a hombres

coef_hombre = prob_hombre_comprar / (1 - prob_hombre_comprar)
coef_hombre

0.9680000000000002

### Las matemáticas detrás de la regresión Logística partiendo de la regresión Lineal

In [36]:
print('La regresión lineal se define según la siguinte fórmula:')
display(Math(r'y = \alpha + \beta \cdot x'))
print('Donde:')
display(Math(r'(x,y)\in[-\infty, +\infty]^2'))

La regresión lineal se define según la siguinte fórmula:


<IPython.core.display.Math object>

Donde:


<IPython.core.display.Math object>

In [37]:
# Transformaciones para pasar de la regresión lineal (que adopta valores de y desde menos infinito a más infinito) a una
# regresión logísitica donde y solo adopta valores de 0 a 1

display(Math(r'P = \alpha + \beta\cdot X'))
display(Math(r'\frac{P}{1-P} = \alpha + \beta\cdot X\in [0,+\infty]'))
display(Math(r' ln(\frac{P}{1-P}) = \alpha + \beta\cdot X'))
display(Math(r'\begin{cases}\frac{P}{1-P}\in[0,1]\Rightarrow ln(\frac{P}{1-P})\in[-\infty,0]\\ \frac{P}{1-P}\in[1,+\infty]\Rightarrow ln(\frac{P}{1-P})\in[0, \infty]\end{cases}'))
display(Math(r' ln(\frac{P}{1-P}) = \alpha + \beta\cdot X'))
display(Math(r' \frac{P}{1-P} = e^{\alpha + \beta\cdot X}'))
display(Math(r' P = \frac{e^{\alpha+\beta\cdot X}}{1+e^{\alpha+\beta\cdot X}}'))
display(Math(r' P = \frac{1}{1+e^{-(\alpha+\beta\cdot X)}}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

* Si a+bX es muy pequeño (negativo), entonces P tiende a 0
* Si a+bX = 0, P = 0.5
* Si a+bX es muy grande (positivo), entonces P tiende a 1

### Regresión logísitica MULTIPLE

In [40]:
display(Math(r' P = \frac{1}{1+e^{-(\alpha+\sum_{i=1}^n\beta_i\cdot x_i)}}'))
display(Math(r' \vec{\beta} = (\beta_1,\beta_2,\cdots,\beta_n)'))
display(Math(r' \vec{X} = (x_1,x_2,\cdots,x_n)'))
display(Math(r' P = \frac{1}{1+e^{-(\alpha+\vec{\beta_i}\cdot \vec{X})}}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>